In [1]:
!pip install scikit-learn pandas numpy

In [2]:
import os
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer

# ─── Configuration ─────────────────────────────────────────────────────────
FOLDS_DIR = "cv_folds"
FEATURE_COLS = [
    "PTS_8_game_avg", "PTS_8_game_slope",
    "REB_8_game_avg", "REB_8_game_slope",
    "AST_8_game_avg", "AST_8_game_slope",
    "opp_E_OFF_RATING", "opp_E_DEF_RATING",
    "opp_E_NET_RATING", "opp_REB_PCT"
]
TARGET_COLS = ["PTS", "REB", "AST"]
N_JOBS = 4

# ─── Hyperparameter grid ──────────────────────────────────────────────────
param_grid = {
    "alpha": [0.1, 1.0, 10.0]
}

# ─── Single-fold runner (returns per-target metrics) ───────────────────────
def run_ridge_fold(fold_idx, alpha):
    train_df = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_train.csv", parse_dates=["GAME_DATE"])
    test_df  = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_test.csv", parse_dates=["GAME_DATE"])

    X_train = train_df[FEATURE_COLS].values
    y_train = train_df[TARGET_COLS].values
    X_test  = test_df[FEATURE_COLS].values
    y_test  = test_df[TARGET_COLS].values

    imputer = SimpleImputer(strategy="mean")
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Fit separate Ridge model for each target
    metrics = {"filters": None, "kernel_size": None, "epochs": None, "fold": fold_idx}
    for i, target in enumerate(TARGET_COLS):
        model = Ridge(alpha=alpha, max_iter=10000)
        model.fit(X_train, y_train[:, i])
        preds = model.predict(X_test)

        metrics[f"{target}_rmse"] = np.sqrt(mean_squared_error(y_test[:, i], preds))
        metrics[f"{target}_mae"]  = mean_absolute_error(y_test[:, i], preds)
        metrics[f"{target}_r2"]   = r2_score(y_test[:, i], preds)
    return metrics

# ─── Run across all folds and alphas ────────────────────────────────────────
all_results = []
for alpha in param_grid["alpha"]:
    fold_metrics = Parallel(n_jobs=N_JOBS)(
        delayed(run_ridge_fold)(fold, alpha) for fold in range(1, 11)
    )
    all_results.extend(fold_metrics)

# ─── Compile and display results ────────────────────────────────────────────
final_df = pd.DataFrame(all_results)
# Ensure column order matches CNN format
ordered_cols = [
    "filters", "kernel_size", "epochs", "fold",
    "PTS_rmse", "PTS_mae", "PTS_r2",
    "REB_rmse", "REB_mae", "REB_r2",
    "AST_rmse", "AST_mae", "AST_r2"
]
final_df = final_df[ordered_cols]
print(final_df.to_string(index=False))


filters kernel_size epochs  fold  PTS_rmse  PTS_mae   PTS_r2  REB_rmse  REB_mae   REB_r2  AST_rmse  AST_mae   AST_r2
   None        None   None     1  7.904988 6.170633 0.243764  3.059435 2.351718 0.189943  2.341168 1.760626 0.377836
   None        None   None     2  6.813485 5.381589 0.387687  2.707284 2.067313 0.415920  2.200291 1.631218 0.430725
   None        None   None     3  6.670330 5.313612 0.382486  2.697219 2.057769 0.425424  1.954339 1.495772 0.482776
   None        None   None     4  6.928086 5.453075 0.397137  2.717364 2.103491 0.445413  2.117925 1.587653 0.430236
   None        None   None     5  6.647304 5.208900 0.428067  2.672619 2.030853 0.474100  2.072648 1.566555 0.471024
   None        None   None     6  6.864136 5.364654 0.427685  2.726663 2.095484 0.434128  2.083773 1.592967 0.457661
   None        None   None     7  6.932673 5.474864 0.370234  2.833398 2.178184 0.385441  1.997096 1.533478 0.477070
   None        None   None     8  7.045843 5.360435 0.400404  2.